In [1]:
import pandas
import os, sys
import openai
from tenacity import (retry, stop_after_attempt, wait_random_exponential)
import tiktoken


In [2]:
openai.api_key = ""

@retry(wait=wait_random_exponential(min=1, max=20), stop=stop_after_attempt(7))
def completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

def ask_chat(system_content,line):
    response = completion_with_backoff(
        # model="text-davinci-003",
        model="gpt-3.5-turbo",
        # model="gpt-3.5-turbo-16k-0613",
        messages=[
            {"role": "system", "content": system_content},
            {"role": "user", "content": line},
        ],
        top_p=0.1,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response

In [10]:
# 只抽关系
system_content = '''
As a Communication Assistant, your task is to identify and extract key knowledge concepts from the provided content and determine the relationships between them.
Be vigilant for any meaningless or irrelevant content. 
Your output should be in json, 
'title':Represents all knowledge concepts involved in the relationships.
'en':The English translation of the 'title'.
'type':Indicates the relationship in Chinese from 'source' to 'target'.
'description':A brief description in Chinese of the relationship from 'source' to 'target'.
All values should preferably be in Chinese, except 'en'.
Your output should follow this format:
{ 
"knowledge_points": [
   {"title": "", "en": ""},
   {"title": "", "en": ""},
   ...
], 
"relations": [
   {"source": "", "target": "", "type": ","description":""}, 
   {"source": "", "target": "", "type": "","description":""}, 
   ...
] 
}
                 '''
user_content = ""

In [11]:
def readFile(path,fileName,length):
    paper = []
    with open(path+fileName, 'r', encoding='utf-8') as f:
        content_lines = f.readlines()
        for i in content_lines:
                temp=i.replace('\n','').replace('。','\n').replace('，',',').replace(' ','')# 去掉空格
                temp=temp.replace('（','(').replace('）',')').replace('；',';').replace('：',':')
                temp=temp.replace('“','"').replace('”','"')
                # 将temp按照\n分割
                for j in temp.split('\n'):
                    if len(j)>0:
                        paper.append(j)
        # content_lines = content_lines[0].split('\n') #
        print(len(content_lines))
    lines=[]
    temp_line=''
    for line in paper:
        if len(temp_line)<length:
            temp_line=temp_line+'\n'+line
        else:
            lines.append(temp_line)
            temp_line=''
    lines.append(temp_line) # 添加最后一行没有到3300的
    print(len(lines))
    return lines

In [12]:
Ipath=r'../data/分节去掉习题和MATLAB/'
inputPath='信号与系统'
# fileName='第10章模拟与数字滤波器.txt'
length=1000
# lines=readFile(Ipath+inputPath+'/',fileName,1000)
time_flag=1
Opath=r'../output/教材知识点提取6.25/'
# outputFile=Opath+inputPath+'/'+fileName
# os.mkdir(Opath+'/'+inputPath)

In [13]:
def handle_text(outputFile,system_content, line): 
    response = ask_chat(system_content, line)
    completion = response['choices'][0]["message"]["content"]
    if int(response['usage']['total_tokens']) < 4096:
        with open(outputFile, 'a+', encoding="utf-8-sig") as f:
            f.write(completion)
    else:
        print("超过4096了")
        temp_lines = line.split("\n")
        length = len(temp_lines)
        temp_line1 = ''
        temp_line2 = ''
        index1 = 0
        for i in temp_lines:
            if index1 < int(length / 2):
                temp_line1 = temp_line1 + '\n' + i
            else:
                temp_line2 = temp_line2 + '\n' + i
            index1 += 1
        handle_text(system_content, temp_line1)
        handle_text(system_content, temp_line2)

In [ ]:
for fileName in os.listdir(Ipath+inputPath):
    lines=readFile(Ipath+inputPath+'/',fileName,1000)
    time_flag=1
    Opath=r'../output/教材知识点提取6.25'
    outputFile=Opath+'/'+inputPath+'/'+fileName
    for line in lines:
        print(time_flag / len(lines))
        handle_text(outputFile,system_content, line)
        time_flag += 1

In [6]:
def handle_text(outputFile,system_content, line): 
    response = ask_chat(system_content, line)
    completion = response['choices'][0]["message"]["content"]
    if int(response['usage']['total_tokens']) < 4096:
        with open(outputFile, 'a+', encoding="utf-8-sig") as f:
            f.write(completion)
    else:
        print("超过4096了")
        temp_lines = line.split("\n")
        length = len(temp_lines)
        temp_line1 = ''
        temp_line2 = ''
        index1 = 0
        for i in temp_lines:
            if index1 < int(length / 2):
                temp_line1 = temp_line1 + '\n' + i
            else:
                temp_line2 = temp_line2 + '\n' + i
            index1 += 1
        handle_text(system_content, temp_line1)
        handle_text(system_content, temp_line2)

time_flag = 0
for line in lines:
    print(time_flag / len(lines))
    handle_text(system_content, line)
    time_flag += 1


/workspaces/codespaces-jupyter/notebooks
1647
27


# 判断先后修

In [3]:
# 判断先后修
system_content = '''
As an assistant in the field of Communication, you need to determine the prerequisite relations of knowledge concepts in the knowledge graph of the communication field from the following.
Prerequisite relations refer to learning objectives or concepts that must be understood before others,
which indicate a temporal or logical sequence between the knowledge concepts. 
The content of the text is a triple of knowledge graphs in the communication field.
'head' denotes the head node, 'tail' denotes the tail node, 
'relationship' denotes the relationship between the head node and the tail node,
and 'description' is a brief description of the relationship. 
Determine whether the 'head', 'tail', 'relationship' and 'description' contain a  prerequisite  relationship, if so, extract the 'source' and 'target'  to the output.
Your output should be in json format and should include the following values:
'source' must be the prerequisite knowledge concept for 'target' (meaning you must learn 'source' in order to learn 'target')
'type' indicating the prerequisite relationship from 'source' to 'target',
and 'reason' providing a simple reason of the prerequisite  relationship between 'source' and 'target'. 
All values must be in Chinese.Your output should follow this format:
{ 
  "relations": [
     {"source": "", "target": "", "type": "", "reason": ""}, 
     {"source": "", "target": "", "type": "", "reason": ""}, 
     ...
  ]
}
                 '''
user_content = ""



In [4]:
def process_jsonfile(filepath):
    entity_list = []
    relation_triple_list = []
    with open(filepath, 'r', encoding='utf-8-sig') as f:
        json_string = f.read()

    json_list = eval(json_string)
    for (i, item) in enumerate(json_list):
        knowledge_points_list = item["knowledge_points"]
        relations_list = item["relations"]
        for knowledge_dict in knowledge_points_list:
            entity_list.append([knowledge_dict["title"], knowledge_dict["en"]])
        for relation_dict in relations_list:
            relation_triple_list.append([relation_dict["source"], relation_dict["type"], relation_dict["target"], relation_dict["description"]])

    new_entity_list = list(set([tuple(t) for t in entity_list])) #去重
    new_relation_triple_list = list(set([tuple(t) for t in relation_triple_list]))
    return new_entity_list, new_relation_triple_list
def handle_text(outputFile,system_content, line): 
    response = ask_chat(system_content, line)
    completion = response['choices'][0]["message"]["content"]
    if int(response['usage']['total_tokens']) < 4096:
        with open(outputFile, 'a+', encoding="utf-8-sig") as f:
            f.write(completion)
    else:
        print("超过4096了")
        temp_lines = line.split("\n")
        length = len(temp_lines)
        temp_line1 = ''
        temp_line2 = ''
        index1 = 0
        for i in temp_lines:
            if index1 < int(length / 2):
                temp_line1 = temp_line1 + '\n' + i
            else:
                temp_line2 = temp_line2 + '\n' + i
            index1 += 1
        handle_text(system_content, temp_line1)
        handle_text(system_content, temp_line2)

In [ ]:
# print(new_relation_triple_list[1])
inputPath="../output/教材知识点提取6.25/数字信号处理"
for filename in os.listdir(inputPath):
    # if filename=='第6章信号波形的估计.txt':
    #     continue
    print(filename)
    (new_entity_list, new_relation_triple_list)=process_jsonfile(inputPath+'/'+filename)
    relation = []
    for i in new_relation_triple_list:
        line = '{source:"'+str(i[0]) + '", target:"'+ str(i[2]) + '", type: "'+ str(i[1]) + '", description:"'+ str(i[3]) + '"}'
        relation.append(line)
    lines=[]
    temp_line=''
    for line in relation:
        if len(temp_line)<300:
            temp_line=temp_line+'\n'+line
        else:
            lines.append(temp_line)
            temp_line=''
    lines.append(temp_line) # 添加最后一行没有到3300的
    print(len(lines))
    time_flag=1
    outputPath='../output/教材关系判定6.25/数字信号处理'
    outputFile= outputPath+'/'+filename
    for line in lines:
        print(time_flag / len(lines))
        handle_text(outputFile,system_content,line)
        time_flag += 1

# 以前的

In [15]:
# 以前的

In [7]:
# 强调先后修V2
system_content = '''
As a helpful assistant, your task is to extract knowledge points and identify prerequisite relations between them in the field of Communication Theory from the given contents. 
In the context of education,
prerequisite relations refer to the specific learning objectives or knowledge points that must be understood or skills that must be acquired before moving on to others.
For instance, learning basic arithmetic operations is a prerequisite for studying algebra. 
You should also be aware that there may be some meaningless gibberish in the contents.
Your output should be in the form of json, 
with the 'title' value representing the knowledge points that appear in the relationship, 
the 'en' value representing the English translation of the 'title', 
and the 'type' value indicating the prerequisite relationship between 'source' and 'target' in the field of Communication Theory and in Chinese. 
The values should preferably be in Chinese, except 'en'.
Your output should follow this format:
{ 
"knowledge_points": [
   {"title": "", "en": ""},
   {"title": "", "en": ""}, 
   ...
], 
"relations": [
   {"source": "", "target": "", "type": ""}, 
   {"source": "", "target": "", "type": ""}, 
   ...
] 
}
'''
user_content = ""

In [7]:
# 强调先后修V3
system_content = '''
As an assistant, you need to extract knowledge points and identify prerequisite relations between them in the field of Communication Theory from the contents. 
Prerequisite relations are learning objectives or knowledge points that must be understood before others. 
For example, basic arithmetic is a prerequisite for algebra. 
Be aware of meaningless gibberish in the contents. 
Your output should be in json, 
with the 'title' value representing the knowledge points that appear in the relationships, 
the 'en' value representing the English translation of the 'title', 
the 'type' value indicating the prerequisite relationship between 'source' and 'target' in Chinese,
the 'description' value is a simple description of the relationship between 'source' and 'target' in Chinese.
All values should preferably be in Chinese, except 'en'.
Your output should follow this format:
{ 
"knowledge_points": [
   {"title": "", "en": ""},
   {"title": "", "en": ""},
   ...
], 
"relations": [
   {"source": "", "target": "", "type": ","description":""}, 
   {"source": "", "target": "", "type": "","description":""}, 
   ...
] 
}
'''
user_content = ""

In [5]:
# 强调先后修V4
system_content = '''
As an assistant in the field of Communication, you need to extract knowledge concepts from the contents provided and identify prerequisite relations between them. 
Prerequisite relations are learning objectives or knowledge concepts that must be understood before others. 
For example, basic arithmetic is a prerequisite for algebra. 
Be aware of meaningless gibberish in the contents. 
Your output should be in json, 
with the 'title' value representing the knowledge concept that appear in the relationships, 
the 'en' value representing the English translation of the 'title', 
the 'type' value indicating the prerequisite relationship in Chinese from 'source' to 'target',
note that the 'typr' must be from 'source' to 'target',if the relationship result is '是', a more detailed relationship is required,
the 'description' value is a simple description in Chinese of the relationship from 'source' to 'target',
All values should preferably be in Chinese, except 'en'.
Your output should follow this format:
{ 
"knowledge_points": [
   {"title": "", "en": ""},
   {"title": "", "en": ""},
   ...
], 
"relations": [
   {"source": "", "target": "", "type": ","description":""}, 
   {"source": "", "target": "", "type": "","description":""}, 
   ...
] 
}
'''
user_content = ""

In [34]:
# 强调先后修V5
system_content = '''
As a Communication Assistant, your task is to identify and extract key knowledge concepts from the provided content and determine the prerequisite relationships between them.
Prerequisite relationships refer to the learning objectives or knowledge concepts that must be understood before others can be grasped.
For instance,basic arithmetic is a prerequisite for algebra.
Be vigilant for any meaningless or irrelevant content. 
Your output should be in json, 
'title':Represents all knowledge concepts involved in the relationships.
'en':The English translation of the 'title'.
'type':Indicates the prerequisite relationship in Chinese from 'source' to 'target','type' should not be 'is'or similar,
'description':A brief description in Chinese of the relationship from 'source' to 'target'.
All values should preferably be in Chinese, except 'en'.
Your output should follow this format:
{ 
"knowledge_points": [
   {"title": "", "en": ""},
   {"title": "", "en": ""},
   ...
], 
"relations": [
   {"source": "", "target": "", "type": ","description":""}, 
   {"source": "", "target": "", "type": "","description":""}, 
   ...
] 
}
'''
user_content = ""

In [25]:
paper = []
# 读取全部文件并且分节
print(os.path.abspath(os.curdir))
with open(r"../data/教材llp/数字信号处理（1-3章）.txt", 'r', encoding='utf-8') as f:
    content_lines = f.readlines()
    for i in content_lines:
            temp=i.replace('\n','').replace('。','\n').replace('，',',').replace(' ','')# 去掉空格
            temp=temp.replace('（','(').replace('）',')').replace('；',';').replace('：',':')
            temp=temp.replace('“','"').replace('”','"')
            # 将temp按照\n分割
            for j in temp.split('\n'):
                if len(j)>0:
                    paper.append(j)
    # content_lines = content_lines[0].split('\n') #
    print(len(content_lines))

lines=[]
temp_line=''
for line in paper:
    if len(temp_line)<2000:
        temp_line=temp_line+'\n'+line
    else:
        lines.append(temp_line)
        temp_line=''
lines.append(temp_line) # 添加最后一行没有到3300的
print(len(lines))
time_flag=1
filename= '../output/教材llp/数字信号处理上v2.txt'

/workspaces/codespaces-jupyter/notebooks
2785
44


In [ ]:
print(lines[0])

In [5]:
def handle_text(system_content, line): 
    response = ask_chat(system_content, line)
    completion = response['choices'][0]["message"]["content"]
    if int(response['usage']['total_tokens']) < 4096:
        with open(filename, 'a+', encoding="utf-8-sig") as f:
            f.write(completion)
    else:
        print("超过4096了")
        temp_lines = line.split("\n")
        length = len(temp_lines)
        temp_line1 = ''
        temp_line2 = ''
        index1 = 0
        for i in temp_lines:
            if index1 < int(length / 2):
                temp_line1 = temp_line1 + '\n' + i
            else:
                temp_line2 = temp_line2 + '\n' + i
            index1 += 1
        handle_text(system_content, temp_line1)
        handle_text(system_content, temp_line2)

time_flag = 0
for line in lines:
    print(time_flag / len(lines))
    handle_text(system_content, line)
    time_flag += 1


0.0
0.008849557522123894
超过4096了
0.017699115044247787
0.02654867256637168
0.035398230088495575
0.04424778761061947
0.05309734513274336
0.061946902654867256
0.07079646017699115
0.07964601769911504
0.08849557522123894
0.09734513274336283
0.10619469026548672
0.11504424778761062
0.12389380530973451
0.13274336283185842
0.1415929203539823
0.1504424778761062
0.1592920353982301
0.168141592920354
0.17699115044247787
0.18584070796460178
0.19469026548672566
0.20353982300884957
0.21238938053097345
0.22123893805309736
0.23008849557522124
0.23893805309734514
0.24778761061946902
0.25663716814159293
0.26548672566371684
0.2743362831858407
0.2831858407079646
0.2920353982300885
0.3008849557522124
0.30973451327433627
0.3185840707964602
0.3274336283185841
0.336283185840708
超过4096了
0.34513274336283184
0.35398230088495575
0.36283185840707965
0.37168141592920356
0.3805309734513274
0.3893805309734513
0.39823008849557523
0.40707964601769914
0.415929203539823
0.4247787610619469
0.4336283185840708
0.4424778761061

In [11]:
# 可能还需要对加教材去掉题目
print(filename)

../output/教材知识点提取6.24//数字信号处理上v1.txt


In [12]:
print(filename)

../output/教材知识点提取6.24//数字信号处理上v1.txt
